# Clonamos el repositorio para obtener los dataSet

In [1]:
!git clone https://github.com/joanby/machinelearning-az.git

Cloning into 'machinelearning-az'...
remote: Enumerating objects: 10497, done.
remote: Total 10497 (delta 0), reused 0 (delta 0), pack-reused 10497
Receiving objects: 100% (10497/10497), 310.48 MiB | 33.85 MiB/s, done.
Resolving deltas: 100% (213/213), done.
Checking out files: 100% (10250/10250), done.


# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [2]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

# Instalar dependendias

In [3]:
!pip install sklearn

# Instalar Theano

In [2]:
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-x8264cfp
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-x8264cfp
  Created wheel for Theano: filename=Theano-1.0.5+4.gc5abd7b69-cp37-none-any.whl size=2668282 sha256=1f832564d3fabebf3f4ce39d86bfb35215e1acb9622515e011bedc5c166e8dec
  Stored in directory: /tmp/pip-ephem-wheel-cache-d7hg3s69/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.5
    Uninstalling Theano-1.0.5:
      Successfully uninstalled Theano-1.0.5


# Instalar Tensorflow y Keras


In [3]:
!pip install keras
!pip install tensorflow

# Redes Neuronales Artificales

# Cómo importar las librerías


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set


In [5]:
dataset = pd.read_csv('/content/machinelearning-az/datasets/Part 8 - Deep Learning/Section 39 - Artificial Neural Networks (ANN)/Churn_Modelling.csv')
#Variables de CreditScore - EstimadSalary
X = dataset.iloc[:, 3:13].values
#Variable dependiente - Exited
y = dataset.iloc[:, 13].values
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Parte 1 - Pre procesado de datos

# Codificar datos categóricos

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

#Transforma la variable categorica Geography a dummy (0 y 1)
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
#Variable Gender
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) 
#Crear columnas por cada variable dummy de geography y eliminar una de las columnas para evitar la multicolinealidad
#Si los paises son 3 y si no pertenece a ninguno de los 2 primeros, se asume que corresponde al 3ro.
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

In [14]:
X

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

# Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Escalado de variables

In [8]:
#Es obligatorio realizar escalamiento de los valores para las redes neuronales.
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Parte 2 - Construir la RNA

# Importar Keras y librerías adicionales


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# Inicializar la RNA

In [10]:
#Clase para iniciar la red reuronal
classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta


In [11]:
#Las variables dependientes son 11 (X).
#units -> cantidad capas ocultas (lo ideal es definirlo con la media, de acuerdo con las variables de entradas y la capa de salida.
#kernel_initializer -> para definir la inicialización de los pesos.
#activation -> función de activación.
#input_dim -> dimensión de entrada (cantidad variables dependientes).
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))

# Añadir la segunda capa oculta

In [12]:
#Agregar la segunda capa oculta y por lo tanto, al setear en 6 la primera capa debe tener el mismo valor.
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

# Añadir la capa de salida

In [13]:
#units -> solo 1 salida, si renueva o no.
#sigmoid -> de acuerdo a probabilidades si renueva o no.
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA

In [14]:
#optimizer -> método de optimización de la RNA
#loss -> función de coste (dependiendo de la salida de la red [binaria/categorización])
#metrics -> lista de métrica que queremos que valide la red.
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento

In [15]:
#batch_size -> tamaño de los bloques para entrenar la red.
#epochs -> número de ciclos para realizar el entrenamiento de la red neuronal.
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.5558 - accuracy: 0.7893
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4288 - accuracy: 0.7985
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4174 - accuracy: 0.8009
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4251 - accuracy: 0.8203
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4260 - accuracy: 0.8211
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4201 - accuracy: 0.8289
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4191 - accuracy: 0.8253
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4130 - accuracy: 0.8327
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4096 - accuracy: 0.8304
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4158 - accura

# Parte 3 - Evaluar el modelo y calcular predicciones finales

# Predicción de los resultados con el Conjunto de Testing

In [21]:
y_pred  = classifier.predict(X_test)
#La salida son probabilidades
y_pred

array([[0.20884776],
       [0.3451407 ],
       [0.1527906 ],
       ...,
       [0.16208684],
       [0.15322945],
       [0.0906553 ]], dtype=float32)

In [22]:
#Luego para la matriz de confusión, solo muestra valores categoricos. Por lo tanto, definimos un umbral (el umbral depende del negocio, Ej: churn de
#un banco, predecir si un cliente puede tener cancér o no, etc.)
y_pred = (y_pred>0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Elaborar una matriz de confusión

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1545,   50],
       [ 261,  144]])